# Result logs cleaning

In [191]:
import os
import re
import pandas as pd
import numpy as np
import json

## Initial Cleaning

In [260]:
def get_start_idx(lines, substring):
    '''
    Obtain the beginning of the cleaning
    '''
    return [line_idx for line_idx, line in enumerate(lines) if substring in line]

### Clean logs of attention head mask

In [276]:
LOGS_PATH = './logs'

def get_sliced_logs(lines):
    '''
    Slice the entire logs of multiple experiemnts into seperate experiements.
    '''
    
    # Step 1: Find the starting point.
    starter_ides = get_start_idx(lines, 'attention_head_mask')
    res = pd.DataFrame({'starter_ides': starter_ides})
    
    # Step 2: Base on all the start point, find the range
    res['ender_ides'] = res.apply(lambda x: x.shift(-1))
    res.iloc[-1, -1] = len(lines) # Upper bound of last row is the length!
    res['ender_ides'] = res['ender_ides'].astype('int')

    # Step 3: Obtain the corresponding lines base on the ranges
    sliced_lines = []
    for index, row in res.iterrows():
        starter_idx = row['starter_ides']
        ender_idx = row['ender_ides']
        sliced_lines.append(lines[starter_idx:ender_idx])
    return sliced_lines
        
def clean_log(logs):
    '''
    Clean the log of 1 experiment
    '''
    def clean_one_line(log_line):
        '''
        Clean 1 line of 1 log
        Examples
        --------
        >>> clean_one_line("00:31:00-INFO:   Batch size = 8")
        ('Batch size', '8')
        '''
        if '-INFO:   ' in log_line:
            result = log_line.split('-INFO:   ')

            if result:
                variable, value = result[1].split(' = ')
                value = re.findall(r'[-+]?(?:\d*\.\d+|\d+)', value)
                if value:
                    value = value[0]
                else:
                    value = None
                return variable, value
    variables = []
    values = []
    for log_line in logs:
        result = clean_one_line(log_line)
        if result:
            variable, value = result
            variables.append(variable)
            values.append(value)
    return variables, values

def get_experiment_result(task):
    with open(f'{LOGS_PATH}/head_pruning/{task}.txt') as f:
        lines = f.readlines()
    dfs = []
    for log in get_sliced_logs(lines):
        experiment = log[0].split(' ')[0]
        parameters = eval(log[1].replace('\n', ''))
        variables, values = clean_log(log[2:])
        # if task == 'rte':
        #     print(task, parameters, variables, values)
        df = pd.DataFrame({'task': task.lower(),
                           'experiments': experiment,
                           'drop_head_at_layer': int(parameters[0]),
                           'drop_head': int(parameters[1]),
                           'variables': variables,
                           'values': values})
        dfs.append(df)
    result = pd.concat(dfs, axis=0, ignore_index=True)
    result.columns.name = None
    result.loc[result['variables'].isin(['acc', 'eval_accuracy']), 'variables'] = 'accuracy'
    return result

def get_baseline_result(task):
    with open(f'{LOGS_PATH}/head_pruning/{task}.txt') as f:
        lines = f.readlines()
    header_start = min(get_start_idx(lines, 'Running evaluation'))
    header_end = min(get_start_idx(lines, 'attention_head_mask'))
    variables, values = clean_log(lines[header_start:header_end])

    df = pd.DataFrame({'task': task,
                       'variables': variables,
                       'values': values})
    return df

In [277]:
experiment_results = []
baseline_results = []
for task in [task.replace('.txt', '') for task in os.listdir(LOGS_PATH + '/head_pruning') if '.ipynb_checkpoints' not in task]:
    experiment_results.append(get_experiment_result(task))
    baseline_results.append(get_baseline_result(task))
    
pd.concat(experiment_results, axis=0).to_csv('logs_cleaned/head_pruning_experiment_results.csv', index=False)
pd.concat(experiment_results, axis=0).to_pickle('logs_cleaned/head_pruning_experiment_results.pickle')
pd.concat(baseline_results, axis=0).to_csv('logs_cleaned/baseline_results.csv', index=False)
pd.concat(baseline_results, axis=0).to_pickle('logs_cleaned/baseline_results.pickle')

### Clean logs of layer pruning

In [278]:
def get_logs_2(lines):
    logs = []
    start_ides = get_start_idx(lines, 'EXPERIMENT')
    for log_idx, strat_idx in enumerate(start_ides):
        if log_idx != len(start_ides) - 1:
            end_idx = start_ides[log_idx+1]
            log = lines[strat_idx:end_idx]
            logs.append(log)
    return logs

def get_experiment_result_2(task):
    with open(f'{LOGS_PATH}/layer_drop/{task}.txt') as f:
        lines = f.readlines()
    dfs = []
    for log in get_logs_2(lines):
        experiment = 'Remove Layers'
        parameter = log[0].split(' remove layers ')[1].replace('\n', '')
        variables, values = clean_log(log[3:])
        df = pd.DataFrame({'task': task.lower(),
                           'experiments': experiment,
                           'parameter': parameter,
                           'variables': variables,
                           'values': values})
        dfs.append(df)
    result = pd.concat(dfs, axis=0, ignore_index=True)
    result.columns.name = None
    return result

experiment_results_2 = []
for task in [task.replace('.txt', '') for task in os.listdir(LOGS_PATH + '/layer_drop') if '.ipynb_checkpoints' not in task]:
    experiment_results_2.append(get_experiment_result_2(task))
pd.concat(experiment_results, axis=0).to_csv('logs_cleaned/layer_drop_results.csv', index=False)
pd.concat(experiment_results, axis=0).to_pickle('logs_cleaned/layer_drop_results.pickle')

## Result re-formatting

[GLUE](https://openreview.net/pdf?id=rJ4km2R5t7https://openreview.net/pdf?id=rJ4km2R5t7)

<img width = "50%" src="https://cdn.mathpix.com/snip/images/pS3Kb2-_3rym-Zd4LhhdPZkqIs7-K1cMmMekf7QQ2HE.original.fullsize.png" />

- Note that at [BERT](https://arxiv.org/abs/1810.04805https://arxiv.org/abs/1810.04805), F1 scores are reported for QQP and MRPC.

    <img width = "50%" src="https://cdn.mathpix.com/snip/images/TyBsRFSkPxAnklR4GijMblC8w8kcwXuTcAIVCqfaPdA.original.fullsize.png" />

In [279]:
benchmark_mapper = pd.DataFrame(
    {'task': ['sst-2', 'rte', 'mrpc', 'wnli', 'sts-b', 'cola'],
     'benchmark': ['accuracy', 'accuracy', 'F-1 score', 'accuracy', 'spearmanr', "Matthew's correlation"]}
)

In [280]:
head_prune = pd.read_pickle('logs_cleaned/head_pruning_experiment_results.pickle')

In [286]:
head_prune.merge(benchmark_mapper, how='inner', on='task') \
    .query('variables == benchmark') \
    .drop(columns=['experiments', 'variables']).to_csv('logs_cleaned/head_prune_core_benchmark.csv', index=False)